# Top-k (no econ filtering)

In [ ]:
import pandas as pd

# file_path = 'data/tables_OZ_geo_5500/processed/regex-pairwise-dataset_num-queries=20_num-pairs=6226_patterns-dict-hash=6dbf9b3ef9568e60cd959f87be7e3b26.csv'
file_path = 'data/tables_OZ_geo_5500/processed/regex-pairwise-dataset_num-queries=20_num-pairs=6226_patterns-dict-hash=6dbf9b3ef9568e60cd959f87be7e3b26.csv'
pairwise_df = pd.read_csv(file_path)

# Rename columns ending in _candidate -> _second and _query -> _first
pairwise_df.columns = [
    col.replace('_candidate', '_second').replace('_query', '_first')
    for col in pairwise_df.columns
]

pairwise_df.head()

,sku_first,description_first,image_url_first,name_first,category_first,схема_first,brand_first,niche_first,seller_first,balance_fbo_first,balance_fbs_first,warehouses_count_first,comments_first,final_price_first,max_price_first,min_price_first,average_price_first,median_price_first,membership_card_price_first,sales_first,revenue_first,revenue_potential_first,revenue_average_first,lost_profit_first,lost_profit_percent_first,url_first,thumb_first,pics_count_first,has_video_first,first_date_first,days_in_website_first,days_in_stock_first,days_with_sales_first,average_if_in_stock_first,rating_first,fbs_first,base_price_first,category_position_first,categories_last_count_first,sales_per_day_average_first,frozen_stocks_first,frozen_stocks_cost_first,frozen_stocks_percent_first,balance_first,image_name_first,name_and_description_first,width_first,height_first,sku_second,description_second,image_url_second,name_second,category_second,схема_second,brand_second,niche_second,seller_second,balance_fbo_second,balance_fbs_second,warehouses_count_second,comments_second,final_price_second,max_price_second,min_price_second,average_price_second,median_price_second,membership_card_price_second,sales_second,revenue_second,revenue_potential_second,revenue_average_second,lost_profit_second,lost_profit_percent_second,url_second,thumb_second,pics_count_second,has_video_second,first_date_second,days_in_website_second,days_in_stock_second,days_with_sales_second,average_if_in_stock_second,rating_second,fbs_second,base_price_second,category_position_second,categories_last_count_second,sales_per_day_average_second,frozen_stocks_second,frozen_stocks_cost_second,frozen_stocks_percent_second,balance_second,image_name_second,name_and_description_second,label
0,1871769771,Представляем вашему вниманию уникальный набор ...,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,"Карты МИРА и РОССИИ настенные политические,160...",Канцелярские товары/Бумажная продукция/Карты мира,FBS,Атлас Принт,Канцелярские товары/Бумажная продукция/Карты мира,ИНТЕРТРЕЙД,0,622,1,1,1530,1759,1511,0.0,1565.5,1421,0,0,0.0,0.0,0.0,0.0,https://www.ozon.ru/context/detail/id/1871769771/,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,6,0,45703,36,0,0,0.0,5.0,1,4425,2079,1,0.0,0,0,0.0,622,7323783851.jpg,"Карты МИРА и РОССИИ настенные политические,160...",160,102,1674648694,Двусторонняя настенная карта с новыми границам...,https://cdn1.ozone.ru/s3/multimedia-q/68729595...,Двухсторонняя политическая карта мира и России...,Канцелярские товары/Бумажная продукция/Карты мира,FBS,АГТ Геоцентр,Канцелярские товары/Бумажная продукция/Карты мира,Store-M,0,2,1,0,3549,5038,3528,0.0,4975.0,3278,0,0,0.0,0.0,0.0,0.0,https://www.ozon.ru/context/detail/id/1674648694/,https://cdn1.ozone.ru/s3/multimedia-q/68729595...,6,0,45535,204,0,0,0.0,0.0,1,6000,1323,1,0.0,0,0,0.0,2,6872959574.jpg,Двухсторонняя политическая карта мира и России...,1
1,1871769771,Представляем вашему вниманию уникальный набор ...,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,"Карты МИРА и РОССИИ настенные политические,160...",Канцелярские товары/Бумажная продукция/Карты мира,FBS,Атлас Принт,Канцелярские товары/Бумажная продукция/Карты мира,ИНТЕРТРЕЙД,0,622,1,1,1530,1759,1511,0.0,1565.5,1421,0,0,0.0,0.0,0.0,0.0,https://www.ozon.ru/context/detail/id/1871769771/,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,6,0,45703,36,0,0,0.0,5.0,1,4425,2079,1,0.0,0,0,0.0,622,7323783851.jpg,"Карты МИРА и РОССИИ настенные политические,160...",160,102,600779875,Двусторонняя настенная карта с новыми границам...,https://cdn1.ozone.ru/s3/multimedia-q/68729595...,Двухсторонняя политическая карта мира и России...,NaN,FBS,АГТ Геоцентр,Канцелярские товары/Бумажная продукция/Карты мира,ГЕО-ТРЕЙД оптовые продажи,0,0,0,34,2379,2436,1707,0.0,2370.0,2203,0,0,0.0,0.0,0.0,0.0,https://www.ozon.ru/context/detail/id/600779875/,https://cdn1.ozone.ru/s3/multimedia-q/68729595...,6,0,44735,1004,0,0,0.0,5.0,1,3630,0,1,0.0,0,0,0.0,0,6872959574.jpg,Двухсторонняя политическая ка

In [77]:
def construct_wide_table(
    df,
    label_col: str,
    top_k: int = None,
    positive_only: bool = True,
    include_urls: bool = False,
    layout: str = 'blocked',
):
    """
    Builds a wide-format table of top‐k matches per Query_SKU.
    """
    rows = []
    # Determine how many slots: either fixed top_k or max found per group
    if top_k is None:
        max_counts = (
            df[df[label_col] == 1]
            .groupby('sku_first')
            .size()
            .max()
        )
        n_slots = int(max_counts)
    else:
        n_slots = int(top_k)

    for query_sku, group in df.groupby('sku_first'):
        candidates = group[group[label_col] == 1] if positive_only else group

        # always reset index so positional iloc works
        candidates = candidates.reset_index(drop=True)

        # then sort by probability if available
        if 'proba' in candidates.columns:
            candidates = (
                candidates.sort_values('proba', ascending=False)
                          .reset_index(drop=True)
            )

        row = {'Query_SKU': int(query_sku)}

        for i in range(n_slots):
            if i < len(candidates):
                row[f'Top-{i+1}_SKU'] = int(candidates.iloc[i]['sku_second'])
                if include_urls and 'url_second' in candidates.columns:
                    row[f'Top-{i+1}_URL'] = candidates.iloc[i]['url_second']
                if 'proba' in candidates.columns:
                    row[f'Top-{i+1}_Proba'] = float(candidates.iloc[i]['proba'])
            else:
                row[f'Top-{i+1}_SKU'] = -1
                if include_urls:
                    row[f'Top-{i+1}_URL'] = ''
                if 'proba' in candidates.columns:
                    row[f'Top-{i+1}_Proba'] = 0.0

        rows.append(row)

    matches_wide_df = pd.DataFrame(rows)

    if layout == 'blocked':
        sku_cols   = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_SKU')]
        url_cols   = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_URL')]
        proba_cols = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_Proba')]
        matches_wide_df = matches_wide_df[['Query_SKU'] + sku_cols + url_cols + proba_cols]

    return matches_wide_df


In [93]:
# get matches for unfiltered data

all_matches_wide_df = construct_wide_table(
    df=pairwise_df,
    label_col = 'label',
    top_k = None,
    positive_only=True,
    include_urls=False,
    layout='blocked',
)
all_matches_wide_df.dropna(inplace=True)
pd.set_option('display.max_columns', None)  # 'max_cols' is also valid
all_matches_wide_df

,Query_SKU,Top-1_SKU,Top-2_SKU,Top-3_SKU,Top-4_SKU,Top-5_SKU,Top-6_SKU,Top-7_SKU,Top-8_SKU,Top-9_SKU,Top-10_SKU,Top-11_SKU,Top-12_SKU,Top-13_SKU,Top-14_SKU,Top-15_SKU,Top-16_SKU,Top-17_SKU,Top-18_SKU,Top-19_SKU,Top-20_SKU,Top-21_SKU,Top-22_SKU,Top-23_SKU,Top-24_SKU,Top-25_SKU,Top-26_SKU,Top-27_SKU,Top-28_SKU,Top-29_SKU,Top-30_SKU,Top-31_SKU,Top-32_SKU,Top-33_SKU,Top-34_SKU,Top-35_SKU,Top-36_SKU,Top-37_SKU,Top-38_SKU,Top-39_SKU,Top-40_SKU,Top-41_SKU,Top-42_SKU,Top-43_SKU,Top-44_SKU,Top-45_SKU,Top-46_SKU,Top-47_SKU,Top-48_SKU,Top-49_SKU,Top-50_SKU,Top-51_SKU,Top-52_SKU,Top-53_SKU,Top-54_SKU,Top-55_SKU,Top-56_SKU,Top-57_SKU,Top-58_SKU,Top-59_SKU,Top-60_SKU,Top-61_SKU,Top-62_SKU,Top-63_SKU,Top-64_SKU,Top-65_SKU,Top-66_SKU,Top-67_SKU,Top-68_SKU,Top-69_SKU,Top-70_SKU,Top-71_SKU,Top-72_SKU,Top-73_SKU,Top-74_SKU,Top-75_SKU,Top-76_SKU,Top-77_SKU,Top-78_SKU,Top-79_SKU,Top-80_SKU,Top-81_SKU,Top-82_SKU,Top-83_SKU,Top-84_SKU,Top-85_SKU,Top-86_SKU,Top-87_SKU,Top-88_SKU,Top-89_SKU,Top-90_SKU,Top-91_SKU,Top-92_SKU,Top-93_SKU,Top-94_SKU,Top-95_SKU,Top-96_SKU,Top-97_SKU,Top-98_SKU,Top-99_SKU,Top-100_SKU,Top-101_SKU,Top-102_SKU,Top-103_SKU,Top-104_SKU,Top-105_SKU,Top-106_SKU,Top-107_SKU,Top-108_SKU,Top-109_SKU,Top-110_SKU,Top-111_SKU,Top-112_SKU,Top-113_SKU,Top-114_SKU,Top-115_SKU,Top-116_SKU,Top-117_SKU,Top-118_SKU,Top-119_SKU,Top-120_SKU,Top-121_SKU,Top-122_SKU,Top-123_SKU,Top-124_SKU,Top-125_SKU,Top-126_SKU,Top-127_SKU,Top-128_SKU,Top-129_SKU,Top-130_SKU,Top-131_SKU,Top-132_SKU,Top-133_SKU,Top-134_SKU,Top-135_SKU,Top-136_SKU,Top-137_SKU
0,491268805,1176719536,804154003,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,491270272,1849926941,1776962648,1757684675,1406174525,1011784724,1005611591,945075481,943273414,855023860,733221245,553009244,376669109,339485530,207940625,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,491270369,1899887107,1863560179,1857130100,1849927001,1819952117,1819952021,1819952017,1770976472,1764021617,1757679586,1747643270,1737112763,1737112217,1679172521,1679157969,1678734996,1640641982,1471552778,1469824975,1438364308,1436512495,1426694661,1314312894,1284969012,1199901842,1179422907,1155711152,1155705296,1125081592,1125016187,1011787987,951285479,945075469,843876035,808691779,805731216,733220476,646886872,613582765,601762703,601727858,601677896,601560985,553026693,552998514,417618668,376761939,376669394,289294807,219077186,207940684,207940668,178726257,166584097,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,491271284,1887692023,1887527156,1881059329,1880357856,1819952045,1798159168,1758403711,852246519,601635285,541972235,518722000,518714248,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,491271320,922229770,1902223868,1902142540,1902142428,1899881517,1859895650,1720888780,1616079834,1565839292,1558840721,1558669721,1158311532

In [89]:
# save matches and pairs for unfiltered data

from pathlib import Path

SAVE_DIR = Path('data/tables_OZ_geo_5500/test_results')
INCLUDE_PAIR_COLS = [
    'sku_first', 'sku_second',
    'name_first', 'name_second',
    'final_price_first', 'final_price_second',
    'balance_fbo_first', 'balance_fbo_second',
    'balance_fbs_first', 'balance_fbs_second',
    'rating_first', 'rating_second',
    'description_first', 'description_second'
]

all_matches_wide_df.to_csv(SAVE_DIR / 'regex-evaluation_all-matches.csv', index=False)
pairwise_df[INCLUDE_PAIR_COLS].to_csv(SAVE_DIR / 'regex-evaluation_all-matches-paired.csv', index=False)

# Top-k w/ econ filtering

In [ ]:
# Filter by economic features

PRICE_MARGIN = 0.3
MIN_SALES = 0
MIN_FBO = 0
MIN_RATING = 4.3

# --- original filtering mask ---
mask = (
    (pairwise_df['sales_second'] > MIN_SALES)
    & ((pairwise_df['balance_fbo_second'] > MIN_FBO) | (pairwise_df['balance_fbs_second'] > MIN_FBO))
    & (pairwise_df['rating_second'] > MIN_RATING)
)
filtered = pairwise_df[mask]

# --- now per‐sku_first price‐ratio filtering ---
def within_margin(group):
    ratio = group['final_price_second'] / group['final_price_first']
    return group[ratio.between(1 - PRICE_MARGIN, 1 + PRICE_MARGIN)]

filtered_pairwise_df = (
    filtered
    .groupby('sku_first', group_keys=False)
    .apply(within_margin)
)

print('Original size:', len(pairwise_df))
print('Filtered size:', len(filtered_pairwise_df))


Original size: 6226
Filtered size: 143


/tmp/ipykernel_4967/1827976136.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(within_margin)


In [102]:
# display & save filtered data

filtered_matches_wide_df = construct_wide_table(
    df=filtered_pairwise_df,
    label_col = 'label',
    top_k = None,
    positive_only=True,
    include_urls=False,
    layout='blocked',
)

# Drop rows with all -1
cols_to_check = list(set(filtered_matches_wide_df.columns) - {'Query_SKU'})
filtered_matches_wide_df = filtered_matches_wide_df[
    ~(filtered_matches_wide_df[cols_to_check] == -1).all(axis=1)
]

pd.set_option('display.max_columns', None)  # 'max_cols' is also valid
filtered_matches_wide_df

,Query_SKU,Top-1_SKU,Top-2_SKU,Top-3_SKU,Top-4_SKU,Top-5_SKU,Top-6_SKU,Top-7_SKU,Top-8_SKU
0,491268805,804154003,-1,-1,-1,-1,-1,-1,-1
1,491270369,1857130100,1737112763,1737112217,219077186,178726257,-1,-1,-1
2,491271284,1758403711,-1,-1,-1,-1,-1,-1,-1
3,491271320,861593242,268682160,-1,-1,-1,-1,-1,-1
4,491271768,1711547819,-1,-1,-1,-1,-1,-1,-1
5,491273438,1317729731,1190097076,-1,-1,-1,-1,-1,-1
6,491273791,1758403630,-1,-1,-1,-1,-1,-1,-1
7,491279127,1713026634,-1,-1,-1,-1,-1,-1,-1
9,508611672,847687475,847684763,-1,-1,-1,-1,-1,-1
10,824158517,922231521,1713036930,861605997,856647774,851295373,671211264,-1,-1


In [103]:
# save matches and pairs for filtered data

from pathlib import Path

SAVE_DIR = Path('data/tables_OZ_geo_5500/test_results')
INCLUDE_PAIR_COLS = [
    'sku_first', 'sku_second',
    'name_first', 'name_second',
    'final_price_first', 'final_price_second',
    'balance_fbo_first', 'balance_fbo_second',
    'balance_fbs_first', 'balance_fbs_second',
    'rating_first', 'rating_second',
    'description_first', 'description_second'
]

filtered_matches_wide_df.to_csv(SAVE_DIR / 'regex-evaluation_filtered-matches.csv', index=False)
filtered_pairwise_df[INCLUDE_PAIR_COLS].to_csv(SAVE_DIR / 'regex-evaluation_filtered-matches-paired.csv', index=False)